# Обработка данных для функциональности "Определение упоминания организаций (брендов)"

Корпус: [Nerus](https://github.com/natasha/nerus)

Формат: CoNLL (BIO)

In [4]:
!pip install nerus

In [10]:
from nerus import load_nerus

In [99]:
NERUS = '/content/drive/MyDrive/thesis_datasets/nerus_lenta.conllu.gz'
SENTENCES_PATH = '/content/drive/MyDrive/thesis_datasets/ner_sentences.txt'
TAGS_PATH = '/content/drive/MyDrive/thesis_datasets/ner_tags.txt'

In [86]:
docs = load_nerus(NERUS)

Для первой версии библиотеки ограничим число документов, на которых будет обучаться NER-модель, до 10000. В дальнейшем можно будет увеличить число документов и обучить модель на бОльшей коллекции.

In [82]:
%%time
count_sents = 0
count_tokens = 0

for _ in range(10000):
    doc = next(docs)
    for sent in doc.sents:
        count_sents+=1
        for token in sent.tokens:
            count_tokens+=1

CPU times: user 16.2 s, sys: 34 ms, total: 16.2 s
Wall time: 16.2 s


In [84]:
count_sents, count_tokens

(119265, 2158824)

In [56]:
# доступ к одному токену одного предложения
doc.sents[0].tokens[0].text

'Хакерская'

In [57]:
# доступ к одному тегу одного токена одного предложения
doc.sents[0].tokens[0].tag

'O'

Составим списки всех предложений и последовательностей тегов и запишем их в отдельные файлы, которые понадобятся для обучения NER-модели

In [87]:
%%time
sentences = []
tags = []

for _ in range(10000):
    doc = next(docs)
    for sent in doc.sents:
        sentence = []
        tags_for_sentence = []
        for token in sent.tokens:
            sentence.append(token.text)
            tags_for_sentence.append(token.tag)
        sentences.append(sentence)
        tags.append(tags_for_sentence)

CPU times: user 17.7 s, sys: 408 ms, total: 18.1 s
Wall time: 18.2 s


In [88]:
len(sentences), len(tags)

(119265, 119265)

In [98]:
for word, tag in zip(sentences[0], tags[0]):
    print((word, tag))

('Вице-премьер', 'O')
('по', 'O')
('социальным', 'O')
('вопросам', 'O')
('Татьяна', 'B-PER')
('Голикова', 'I-PER')
('рассказала', 'O')
(',', 'O')
('в', 'O')
('каких', 'O')
('регионах', 'O')
('России', 'B-LOC')
('зафиксирована', 'O')
('наиболее', 'O')
('высокая', 'O')
('смертность', 'O')
('от', 'O')
('рака', 'O')
(',', 'O')
('сообщает', 'O')
('РИА', 'B-ORG')
('Новости', 'I-ORG')
('.', 'O')


Для первой версии нужно оставить только теги организаций, поскольку библиотека должна будет распознавать только названия организаций и брендов.

In [105]:
cleaned_sentences = []
cleaned_tags = []

for sentence, tag_sequence in zip(sentences, tags):
    cleaned_sentence = []
    cleaned_tag = []
    for word, tag in zip(sentence, tag_sequence):
        if tag != 'B-ORG' and tag != 'I-ORG' and tag != 'O':
            tag = 'O'
        else:
            tag = tag
        cleaned_sentence.append(word)
        cleaned_tag.append(tag)
    cleaned_sentences.append(cleaned_sentence)
    cleaned_tags.append(cleaned_tag)

In [106]:
for word, tag in zip(cleaned_sentences[0], cleaned_tags[0]):
    print((word, tag))

('Вице-премьер', 'O')
('по', 'O')
('социальным', 'O')
('вопросам', 'O')
('Татьяна', 'O')
('Голикова', 'O')
('рассказала', 'O')
(',', 'O')
('в', 'O')
('каких', 'O')
('регионах', 'O')
('России', 'O')
('зафиксирована', 'O')
('наиболее', 'O')
('высокая', 'O')
('смертность', 'O')
('от', 'O')
('рака', 'O')
(',', 'O')
('сообщает', 'O')
('РИА', 'B-ORG')
('Новости', 'I-ORG')
('.', 'O')


In [108]:
with open(SENTENCES_PATH, "w") as sentfile:
    for sentence in cleaned_sentences:
        for token in sentence:
            sentfile.write(token + "\n")
        sentfile.write("\n" + "---------------" + "\n")


with open(TAGS_PATH, "w") as tagfile:
    for tag_sequence in cleaned_tags:
        for token in tag_sequence:
            tagfile.write(token + "\n")
        tagfile.write("\n" + "---------------" + "\n")

In [109]:
with open(SENTENCES_PATH, "r") as sentfile:
    sentences = sentfile.read().split("---------------")

with open(TAGS_PATH, "r") as tagfile:
    tags = tagfile.read().split("---------------")

In [ ]:
sentences = [sent.split() for sent in sentences]
tags = [tag.split() for tag in tags]

In [115]:
for word, tag in zip(sentences[0], tags[0]):
    print((word, tag))

('Вице-премьер', 'O')
('по', 'O')
('социальным', 'O')
('вопросам', 'O')
('Татьяна', 'O')
('Голикова', 'O')
('рассказала', 'O')
(',', 'O')
('в', 'O')
('каких', 'O')
('регионах', 'O')
('России', 'O')
('зафиксирована', 'O')
('наиболее', 'O')
('высокая', 'O')
('смертность', 'O')
('от', 'O')
('рака', 'O')
(',', 'O')
('сообщает', 'O')
('РИА', 'B-ORG')
('Новости', 'I-ORG')
('.', 'O')
